In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma
import chromadb
from pathlib import Path
import os


# Load the CSV file
file_path = "Reduced_LLM_KB.csv"
loader = CSVLoader(file_path=file_path)
data = loader.load()
data = data[:4]


In [2]:
#Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=64)
chunked_docs = text_splitter.split_documents(data)

In [3]:
# Split the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=64)
chunked_docs = text_splitter.split_documents(data)

In [4]:
# Print the number of unique split documents
s = set()
for doc in chunked_docs:
    s.add(doc.page_content)
print("Number of unique split documents:", len(s))

Number of unique split documents: 16


In [5]:
# Set the absolute path for the Chroma database
ABS_PATH = Path().resolve().joinpath('Chroma1')
DB_DIR = os.path.join(ABS_PATH, "env_policy")
print('DB_DIR : ', DB_DIR)

DB_DIR :  /Users/rahul/Desktop/AI-Project/ClimatePolicyRAG/Chroma1/env_policy


In [6]:
# Define client settings for Chroma
client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory=DB_DIR,
    anonymized_telemetry=False,
)

In [7]:
# Initialize the HuggingFaceBgeEmbeddings with the correct model
embedder = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Embed the split documents
embedded_docs = embedder.embed_documents([doc.page_content for doc in chunked_docs])


In [9]:
# Create the Chroma vector store from the embedded documents
bge_vectorstore = Chroma.from_documents(
    documents=chunked_docs,
    embedding=embedder,
    client_settings=client_settings,
    collection_name="env_policy_bge",
    collection_metadata={"hnsw": "cosine"}
)


In [10]:
retriever = bge_vectorstore.as_retriever(
                                  search_type="mmr",
                                  search_kwargs={"k": 5, "include_metadata": True}
                                  )
                                  

In [11]:
query = "Explain climate policy"
retreived_docs = retriever.invoke(query)
retreived_docs

Number of requested results 20 is greater than number of elements in index 16, updating n_results = 16


[Document(page_content='Content: city around world building better financial strategy . heart transition lie strategic reallocation investment away high-emitting activity clean energy climate-friendly industry . move prudent financial standpoint , well aligned climate goal – investment portfolio excluding fossil fuel delivered superior return past decade , fossil fuel investment increasingly risky.1 article set city get started moving investment manage risk grasp opportunity presented transition low carbon economy . look reallocation investment held city employee , example city pension fund . increase fund delivery city ’ climate measure , part shift support sustainable industry large . read 6 way drive climate investment route paying city ’ climate project , programme investment . make public commitment divest fossil fuel scale investment sustainable solution political commitment city ’ senior leadership essential , sending strong message asset manager stakeholder generating public in